**Author**: Dr. Purna Gamage

**Download**: [Click here](lab-5.2-RF.zip)

**Instructions**

* Read and work through the demonstration content on your own
* Complete all exercises below for the `R part` in a `.rmd` notebook 

**Submission:**

* You need to upload ONE document to Canvas when you are done. 
  * A PDF (or HTML) of the completed form of the R lab-assignment notebook
* The final uploaded version should NOT have any code-errors present. 
* All outputs must be visible in the uploaded version, including code-cell outputs, images, graphs, etc


# Demonstration


In [ ]:
library('sparkline')

### Classification trees 

#### Setup


In [ ]:
library(tidyverse)
library(dplyr)
library(tree)
library(ISLR)
library(caret)
library(rpart)
library(rattle)
library(visNetwork)

In [ ]:
attach(Carseats)
hist(Carseats$Sales)

Carseats <- Carseats %>% mutate(High = if_else(Sales <= 8, 0, 1) %>% as.factor())

#### Initial tree


In [ ]:
# tree.carseats=tree::tree(High~.,data=Carseats %>% select(-Sales))
tree.carseats=tree::tree(High~.,data=Carseats)

summary(tree.carseats)
plot(tree.carseats)
text(tree.carseats,pretty=0) #annotate

#### Train/test split


In [ ]:
set.seed(2)
train=sample(1:nrow(Carseats), 200)
Carseats.test=Carseats[-train,]
tree.carseats=tree(High~.-Sales,Carseats,subset=train)
tree.pred=predict(tree.carseats,Carseats.test,type="class")
confusionMatrix(tree.pred,Carseats.test$High)

#### Pruning the tree using CV


In [ ]:
set.seed(3)
cv.carseats=cv.tree(tree.carseats,FUN=prune.misclass)
names(cv.carseats)
cv.carseats
par(mfrow=c(1,2))
plot(cv.carseats$size,cv.carseats$dev,type="b")
plot(cv.carseats$k,cv.carseats$dev,type="b")
prune.carseats=prune.misclass(tree.carseats,best=9)
plot(prune.carseats)
text(prune.carseats,pretty=0)
tree.pred=predict(prune.carseats,Carseats.test,type="class")
confusionMatrix(tree.pred,Carseats.test$High)

#### Pruning the tree manually


In [ ]:
prune.carseats=prune.misclass(tree.carseats,best=15)
plot(prune.carseats)
text(prune.carseats,pretty=0)
tree.pred=predict(prune.carseats,Carseats.test,type="class")
confusionMatrix(tree.pred,Carseats.test$High)

### Fitting Regression Trees 

#### Initial tree - using rpart and rattle / visNetwork


In [ ]:
library(MASS)
set.seed(1)
train = sample(1:nrow(Boston), nrow(Boston)/2)
tree.boston=tree(medv~.,Boston,subset=train)
summary(tree.boston)
tree.boston2=rpart(medv~.,Boston,subset=train)
fancyRpartPlot(tree.boston2)
visTree(tree.boston2)

#### Pruning tree - using rpart and visNetwork


In [ ]:
cv.boston=cv.tree(tree.boston)
plot(cv.boston$size,cv.boston$dev,type='b')
prune.boston=prune.tree(tree.boston,best=5)
plot(prune.boston)
text(prune.boston,pretty=0)
yhat=predict(tree.boston,newdata=Boston[-train,])
boston.test=Boston[-train,"medv"]
plot(yhat,boston.test)
abline(0,1)
mean((yhat-boston.test)^2)

### Bagging and Random Forests 

#### Bagging mtry = # variables

![](rf-meme.png)


In [ ]:
library(randomForest)
set.seed(1)
bag.boston=randomForest(medv~.,data=Boston,subset=train,mtry=13,importance=TRUE)
bag.boston
yhat.bag = predict(bag.boston,newdata=Boston[-train,])
plot(yhat.bag, boston.test)
abline(0,1)
mean((yhat.bag-boston.test)^2)

#### Adjust number of trees


In [ ]:
bag.boston=randomForest(medv~.,data=Boston,subset=train,mtry=13,ntree=25)
yhat.bag = predict(bag.boston,newdata=Boston[-train,])
mean((yhat.bag-boston.test)^2)

#### Random forest mtry < # variables


In [ ]:
set.seed(1)
rf.boston=randomForest(medv~.,data=Boston,subset=train,mtry=6,importance=TRUE)
yhat.rf = predict(rf.boston,newdata=Boston[-train,])
mean((yhat.rf-boston.test)^2)
importance(rf.boston)
varImpPlot(rf.boston)

### Boosting


In [ ]:
library(gbm)
set.seed(1)
boost.boston=gbm(medv~.,data=Boston[train,],distribution="gaussian",n.trees=5000,interaction.depth=4)
summary(boost.boston)
par(mfrow=c(1,2))
plot(boost.boston,i="rm")
plot(boost.boston,i="lstat")
yhat.boost=predict(boost.boston,newdata=Boston[-train,],n.trees=5000)
mean((yhat.boost-boston.test)^2)
boost.boston=gbm(medv~.,data=Boston[train,],distribution="gaussian",n.trees=5000,interaction.depth=4,shrinkage=0.2,verbose=F)
yhat.boost=predict(boost.boston,newdata=Boston[-train,],n.trees=5000)
mean((yhat.boost-boston.test)^2)

# Lab Assignment

### Problem 1: 

Implement decision tree from scratch

a) _Warm up_. Write a function called rss() that takes in a vector and returns the residual sum of squares (relative to the mean of the vector).


In [ ]:
rss <- function(v) {
  mean_v <- mean(v)
  res <- v - mean_v
  rss <- sum((res)^2)
  
  return(rss)
}

b) _Best Split_. We're next going to write a function best_split() that will form the basis of a decision tree. This function will take in two arguments: a vector predictor variable (i.e., x) and a vector target variable (i.e., y). This function will identify the single location in the x domain that would yield the best split of the data, such that the two halves now each have least total RSS in the target variable. 


In [ ]:
best_split <- function(x, y) {
  n <- length(x)
  split_locations <- numeric(n - 1)
  improvements <- numeric(n - 1)

  for (i in 1:(n - 1)) {
    split_value <- (x[i] + x[i + 1]) / 2
    left_idx <- x <= split_value
    right_idx <- x > split_value

    rss_left <- rss(y[left_idx])
    rss_right <- rss(y[right_idx])

    new_rss <- rss_left + rss_right
    old_rss <- rss(y)
    improvement <- old_rss - new_rss

    split_locations[i] <- split_value
    improvements[i] <- improvement
  }

  best_split_idx <- which.max(improvements)
  best_split_loc <- split_locations[best_split_idx]

  left_mean <- mean(y[x <= best_split_loc])
  right_mean <- mean(y[x > best_split_loc])
  
  cat(
    "Split Location:", best_split_loc, "\n",
    "Left Mean:", left_mean, "\n",
    "Right Mean:", right_mean, "\n",
    "Improvement:", improvements[best_split_idx], "\n"
  )
}

Some things to think about:
 (i) If your input x vector has *n* data points, how many possible split locations are there? 
 (ii) You can accomplish this task by brute force. For every possible split location, split the data into two parts and compute the new total RSS. Then just return whichever split location was the optimal one.
 
Make sure your function returns a few things (perhaps in a list): the location of the x split, the mean of y for each of the split parts, the improvement in RSS that was achieved by the split. 

c) _One Dimensional Data_ Here is a synthetic data set with one predictor and one response. Use your function to find out where the first split would be if $y$ is predicted from $x$ with a regression tree.


In [ ]:
x = seq(0,10,by = .01)
y0 = cos(x/4 + x^2/5)*x^2/20 + x 
y = y0 + rnorm(length(x))
mydf = data.frame(x=x,y=y)
rss0 = 1000*var(y)
plot(x,y)
split0 <- best_split(x,y) 
rss0

What is the total RSS of y? What this RSS reduced to when you split the data? 

The question is a little ambiguous to me, but the total RSS of y is observed as approximately 11200, and the improvement in RSS is observed as approximately 6200. The RSS reduced the value of improvement as the tree was split.

d) _Growing the Tree_ 
Split the lower half again. Split the upper half again. What is the total RSS now?


In [ ]:
subset_data <- function(x, y, condition) {
  list(
    x = x[condition],
    y = y[condition]
  )
}

cat("\nInitial Split\n")

best_split2 <- function(x, y, depth = 1) {
  n <- length(x)
  split_locations <- numeric(n - 1)
  improvements <- numeric(n - 1)

  for (i in 1:(n - 1)) {
    split_value <- (x[i] + x[i + 1]) / 2
    left_idx <- x <= split_value
    right_idx <- x > split_value

    rss_left <- rss(y[left_idx])
    rss_right <- rss(y[right_idx])

    new_rss <- rss_left + rss_right
    old_rss <- rss(y)
    improvement <- old_rss - new_rss

    split_locations[i] <- split_value
    improvements[i] <- improvement
  }

  best_split_idx <- which.max(improvements)
  best_split_loc <- split_locations[best_split_idx]

  left_mean <- mean(y[x <= best_split_loc])
  right_mean <- mean(y[x > best_split_loc])

  cat(
    paste0("Split Location (Depth ", depth, "): ", best_split_loc, "\n"),
    paste0("Left Mean (Depth ", depth, "): ", left_mean, "\n"),
    paste0("Right Mean (Depth ", depth, "): ", right_mean, "\n"),
    paste0("Improvement (Depth ", depth, "): ", improvements[best_split_idx], "\n")
  )


  if (depth < 2) {
    left_branch <- subset_data(x, y, x <= best_split_loc)
    right_branch <- subset_data(x, y, x > best_split_loc)

    cat("\nLower Branch (Depth ", depth + 1, "):\n")
    best_split2(left_branch$x, left_branch$y, depth + 1)

    cat("\nUpper Branch (Depth ", depth + 1, "):\n")
    best_split2(right_branch$x, right_branch$y, depth + 1)
  }
  cat("\n", new_rss, "\n")
}

split1 <- best_split2(x, y)
split1

The total RSS for this function is approximately 11000, and it is observed that the value of improvement has decreased significantly in the lower branch, but increased again in the upper branch.  

e) Try this all again with the same data but with R's tree() function. Make a plot of the tree. Do the first few splits agree with the results of your method?


In [ ]:
tree_model <- tree(y~x, mydf)
plot(tree_model)
text(tree_model, pretty = 0)

f) Now that we have a good understanding of how to do recursive binary splitting of a single variable, how do would you handle multiple predictors? 

When there are multiple predictors, binary splitting method is the best method for every predictor and choosing the lowest RSS value is critical when choosing the split point. 

### Problem 2: 

**Complete one of two problems on abalone shells or airfoil data**

Your group will be assigned one of the following problems to go over during lab. You will walk through your results at the end of class

#### 2a

_Work with Abalone Data_. These data may be found on the [UC Irvine website](http://archive.ics.uci.edu/ml/). They give various physical characteristics of about 4000 abalone shellfish. The data were collected in Tasmania in 1995. Make sure that the data are in your source directory.


In [ ]:
library(MASS)
library(ggplot2)

a) We'll try to predict the number of Rings, using the other features. Train a linear model as a baseline case. What is the RSS or Rsquared?


In [ ]:
load("abalone.RData")
names(abalone)
nrow(abalone)
# str(abalone)

In [ ]:
# linear model
model1 = lm(Rings~., data=abalone)
summary(model1)$r.squared # r-squared
sum(model1$residuals^2) # rss

R-squared value is 0.5378844 and RSS value is 20060.73. 

b) Now build a tree and plot it.  We can make the annotation of the tree look better by reducing the font size with the \texttt{cex} parameter. What is the depth of the tree? How many leaves does it have?


In [ ]:
tree_model <- tree(Rings~., data = abalone)
plot(tree_model, cex = .5)
text(tree_model, pretty = 0, cex = .5)

The depth of the tree is 4, and the leaves are 10. 


c) We can manually prune the tree to whatever depth we want. Use the function prune.tree() to simplify the tree so it only has 4 leaves. Visualize this tree.


In [ ]:
pruned_tree <- prune.tree(tree_model, best = 4)

plot(pruned_tree)
text(pruned_tree, pretty = 0, cex = 0.5)

d) Which two continuous predictors seem to be highly predictive according to the tree? Draw a sketch of the feature space and the splits in the space, as well as the predicted number of Rings for each region.

Based on the tree plot, the two continuous predictors that are highly predictive are __Shell Weight__ and __Shucked Weight__.


In [ ]:
pruned_tree <- prune.tree(tree_model, best = 10)

plot(pruned_tree)
text(pruned_tree, pretty = 0, cex = 0.5)

for (leaf in 1:length(pruned_tree$frame$yval)) {
  predicted_value <- pruned_tree$frame$yval[leaf]
  text(leaf, predicted_value, label = round(predicted_value, 2), pos = 2, cex = 0.8)
}

e) Decision trees have high variance. Split the Abalone data in half and train two trees (and don't worry about any extra pruning). Observe the differences between them, visualize the two different trees.


In [ ]:
set.seed(1)
row.number <- sample(1:nrow(abalone), 0.5*nrow(abalone))
left = abalone[row.number,]
right = abalone[-row.number,]

par(mfrow=c(1,2))
tree_model1 <- tree(Rings~., data = left)
plot(tree_model1, cex = .5)
text(tree_model1, pretty = 0, cex = .5)

tree_model2 <- tree(Rings~., data = right)
plot(tree_model2, cex = .5)
text(tree_model2, pretty = 0, cex = .5)

The plots are pretty similar to each other, but they are slightly different in terms of the number of leaves, and splitting points. 

#### 2b

Let's explore the airfoil data. We're going to try to predict the "Pressure" feature from the other features. Use one of the tree aggregation methods we have learned about (Random Forest, Boosting, Bagging)

(a) Start with some exploratory visualizations to see how the other features are related to Pressure. Feel free to use pairs(), or scatterplots, or boxplots. Do any features seem to be strongly predictive of Pressure?

(b) Create a train-test split of the data.

(c) Fit a linear model, observe the $R^2$ on the test set.

(d) Fit a single decision tree, one with large depth. Observe the $R^2$ on the test set. Make a plot of three if you wish.

(e) Fit a tree aggregation model, observe the $R^2$ on the test set.
